Preparing the data

In [101]:
import numpy as np
import pandas as pd
data = pd.read_csv("data425288.csv")

T = 25
N = data.ID.max()

y = np.log(data.Sales.values.reshape((N,T)))

X = np.ones((T,2))
X[:,1] = np.log(data.Price[:T])

In [123]:
#test params
K = 3

theta = np.random.rand(K,2) 
pi = np.ones(K)/K

Log-likelihood evaluation

In [201]:
from scipy.stats import norm

def LogL(theta, pi, y, X):
    mu = np.dot(X,theta.T)
    mu = np.repeat(mu[np.newaxis, :, :], N, axis=0)  #(500,25,3)
    y = np.repeat(y[:, :, np.newaxis], K, axis=2)  #(500,25,3)
    
    #pdfs
    probs = norm.pdf(y,mu,1) #(500,25,3)
    
    #prod^T probs
    segments = np.prod(probs, axis=1) #(500,3)
    
    #sum^K pi_c prod^T probs
    combined = np.dot(segments, pi) #(500,)
    
    #sum^N log sum^K pi_c prod^T probs
    LogL = np.log(combined).sum() #(1,)
    return LogL
    
# likelihood = LogL(theta, pi, y, X)
# display(likelihood.shape)
# display(likelihood)

C:\Users\Tim\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  


()

-inf

In [194]:
np.random.seed(1234)
matrix = np.random.randint(1,6,(5,20,3))
print(matrix)

prod = np.prod(matrix, axis=1)
print(prod)

np.dot(prod, pi)

[[[4 5 5]
  [1 2 2]
  [2 3 4]
  [5 5 3]
  [3 1 1]
  [5 1 2]
  [3 1 4]
  [5 3 3]
  [4 4 1]
  [2 4 1]
  [4 3 4]
  [5 2 4]
  [4 4 3]
  [2 4 5]
  [3 4 5]
  [2 5 1]
  [5 2 2]
  [5 2 5]
  [1 2 1]
  [1 5 1]]

 [[5 3 4]
  [2 3 3]
  [5 4 5]
  [3 3 5]
  [5 3 2]
  [4 3 2]
  [5 2 1]
  [2 3 2]
  [3 2 3]
  [2 2 5]
  [2 3 5]
  [1 5 2]
  [3 2 2]
  [4 5 2]
  [4 3 1]
  [5 4 5]
  [1 1 5]
  [1 5 2]
  [1 2 1]
  [3 1 4]]

 [[3 3 4]
  [3 2 5]
  [1 2 2]
  [3 3 5]
  [4 3 5]
  [2 1 1]
  [4 5 1]
  [1 2 1]
  [4 2 2]
  [2 4 4]
  [2 4 5]
  [5 5 2]
  [4 2 3]
  [2 1 1]
  [5 5 1]
  [1 2 4]
  [2 5 2]
  [4 3 4]
  [3 3 1]
  [2 2 4]]

 [[4 5 4]
  [3 5 3]
  [2 4 3]
  [1 2 2]
  [4 5 4]
  [5 3 4]
  [2 1 1]
  [1 2 3]
  [4 2 5]
  [4 2 1]
  [2 5 3]
  [4 2 3]
  [2 5 4]
  [1 5 4]
  [5 1 1]
  [1 2 4]
  [5 1 2]
  [2 1 5]
  [2 2 2]
  [3 4 5]]

 [[3 4 2]
  [2 5 1]
  [1 3 2]
  [5 3 4]
  [3 2 1]
  [1 3 1]
  [1 2 2]
  [5 4 3]
  [1 1 3]
  [1 5 1]
  [4 3 5]
  [5 3 4]
  [5 3 1]
  [3 2 2]
  [1 1 1]
  [2 5 3]
  [2 2 5]
  [2 2 3]
  [5 4 2]
  

array([7.718400e+08, 3.223680e+08, 1.581568e+08, 3.883520e+08,
       2.516688e+08])